In [1]:
import pandas as pd

from sklearn.linear_model import LinearRegression

In [59]:
df_all = pd.read_csv('../data/processed/0.1-initial-exploration.csv',index_col=False)
df_date_filtered = df_all[df_all['year'] > 2005]

In [60]:
df = df_date_filtered[['gridStart','year','round','positionOrder']]

In [61]:
df_train=df[(df['year'] <= 2021) & (df['round']<19)]
df_validation=df[(df['year'] == 2021) & (df['round']==19)]

In [62]:
reg = LinearRegression().fit(df_train.iloc[:,:-3], df_train.iloc[:,-1:])
reg.score(df_train.iloc[:,:-3], df_train.iloc[:,-1:])

0.34854908299384657